In [2]:
import pandas as pd
melb_data = pd.read_csv('data/melb_data_fe.csv', sep=',')
melb_df = melb_data.copy()

In [ ]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
print(melb_df['Date'].dt.quarter.value_counts())

In [ ]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())

In [ ]:
melb_df.sort_values(by='Price').head(10)
melb_df.sort_values(by='Date', ascending=False)
#сортировка сначала по расстоянию, затем по цене и вывод только двух столбцов
#с шагом 10
melb_df.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']]

In [ ]:
#вначале по возрастанию ст. дата, затем по убыванию ст. коэффициент
#индексы на новые, вывод только 2-х столбцов 
mask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]

In [7]:
# чтобы сочетание маски, сортировки и присваивания индексов работало без предупреждения
# копирование столбца с фильтрацией
filtered = melb_df[melb_df['Rooms'] > 5].copy()
filtered.sort_values(inplace=True, by=['Rooms'])

In [8]:
# Задание 2.2
melb_df.sort_values(by=['AreaRatio'], ascending=False, ignore_index=True).loc[1558, ['BuildingArea']]

BuildingArea    126.0
Name: 1558, dtype: object

In [9]:
# Задание 2.3
mask1 = melb_df['Type'] == 'townhouse'
mask2 = melb_df['Rooms'] > 2
filtered = melb_df[mask1 & mask2].copy()
filtered.sort_values(
    inplace=True,
    by=['Rooms', 'MeanRoomsSquare'],
    ascending=[True, False],
    ignore_index = True)
print(filtered['Price'][18])

1300000.0


In [ ]:
# таблица со средними значениями с группировкой по типу
melb_df.groupby(by='Type').mean(numeric_only=True)

In [ ]:
# вывод только столбца с ценой
melb_df.groupby('Type')['Price'].mean()

In [ ]:
# минимальное расстояние от центра до объекта в зависимости от региона
# с сортировкой по убыванию
melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

In [ ]:
# таблица продаж по месяцам, кол-во средняя
# и максимальная цена, группировка по меcяцу продажи, по убыванию
melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)

In [ ]:
# все статистические методы
melb_df.groupby('MonthSale')['Price'].agg('describe')

In [ ]:
# уникальные значения названий компаний и все их названия
melb_df.groupby('Regionname')['SellerG'].agg(
    		['nunique', set]
)

In [ ]:
# Задание 3.1
melb_df.groupby('Rooms')['Price'].mean().sort_values(ascending=False)

In [ ]:
# Задание 3.2
melb_df.groupby('Regionname')['Lattitude'].std().sort_values()

In [ ]:
# Задание 3.3 временной период
date1 = pd.to_datetime('2017-05-01')
date2 = pd.to_datetime('2017-09-01')
mask = (date1 <= melb_df['Date']) & (melb_df['Date']<= date2)
melb_df[mask].groupby('SellerG')['Price'].sum().sort_values()

In [ ]:
#  таблица, которая показывает зависимость медианной цены и площади здания от числа комнат
melb_df.groupby('Rooms')[['Price', 'BuildingArea']].median()

In [ ]:
# учитываются количество комнат и тип дома
melb_df.groupby(['Rooms', 'Type'])['Price'].mean()
# формирует таблицу, кол-во комнат - строки, тип - столбцы
melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

In [ ]:
melb_df.pivot_table(values='Price', index='Rooms', columns='Type', fill_value=0).round()
melb_df.pivot_table(values='Price', index='Regionname', columns='Weekend', aggfunc='count')
melb_df.pivot_table(values='Landsize', index='Regionname', columns='Type',
    aggfunc=['median', 'mean'], fill_value=0)
melb_df.pivot_table(values='Price', index=['Method','Type'], columns='Regionname',
    aggfunc='median', fill_value=0)

In [ ]:
pivot = melb_df.pivot_table(values='Landsize', index='Regionname', columns='Type',
    aggfunc=['median', 'mean'], fill_value=0)
display(pivot['mean']['unit'])
# регионы, в которых средняя площадь здания для домов типа house меньше их медианной площади
mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
display(filtered_pivot)
# получить индексы отфильтрованной таблицы
print(list(filtered_pivot.index))
# ['Southern Metropolitan', 'Western Metropolitan']

In [ ]:
# Задание 4.2
melb_df.pivot_table(values='BuildingArea', index='Type', columns='Rooms',
    aggfunc='median', fill_value=0)

In [ ]:
# Задание 4.3
pivot = melb_df.pivot_table(values='Price', index='SellerG', columns='Type',
    aggfunc='median', fill_value=0)
max_value = pivot['unit'].max()
print(pivot[pivot['unit'] == max_value].index[0]) #выведение названия
